In [1]:

import hashlib
from copy import copy
from dataclasses import dataclass
from pathlib import Path
from time import sleep

import cv2
import mxnet as mx
import numpy as np
import pandas as pd
from gluoncv import utils
from gluoncv.data.transforms import video
from gluoncv.data.transforms.pose import detector_to_alpha_pose, heatmap_to_coord_alpha_pose
from gluoncv.model_zoo import get_model
from gluoncv.utils.filesystem import try_import_decord
from mxnet import nd
from tqdm import tqdm

generated_colors_memory = {}


def id_to_random_color(any, normalized=False):
    number = str(any)
    if not number in generated_colors_memory:
        numByte = str.encode(number)
        hashObj = hashlib.sha1(numByte).digest()
        r, g, b = hashObj[-1], hashObj[-2], hashObj[-3]
        if normalized:
            r = r / 255
            g = g / 255
            b = b / 255
        generated_colors_memory[number] = (r, g, b)
        return r, g, b
    else:
        return generated_colors_memory[number]


def draw_rectangle(image, x1, y1, w, h, label):
    # For bounding box
    img = copy(image)
    color = id_to_random_color(label)
    text_color = (0, 0, 0)
    x2 = x1 + w
    y2 = y1 + h

    img = cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    # For the text background
    # Finds space required by the text so that we can put a background with that amount of width.
    (w, h), _ = cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

    # Prints the text.
    img = cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), color, -1)
    img = cv2.putText(img, str(label), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)

    # For printing text
    # img = cv2.putText(img, 'test', (x1, y1),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
    return img


def view_image(img):
    cv2.imshow('image', img)
    # cv2.waitKey(0)


@dataclass
class Label:
    row_id: int
    frame: int
    person_id: int
    x1: int
    y1: int
    w: int
    h: int


# decord = try_import_decord()
gpu = mx.gpu()


/home/rafa/anaconda3/envs/sportstrack/lib/python3.7/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.8.1+cu111` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [2]:
img_path = r'/home/rafa/PycharmProjects/detector-preview/data/v_i2_L4qquVg0_c009/img1/{frame:06d}.jpg'
gt_path = r'/home/rafa/PycharmProjects/detector-preview/data/v_i2_L4qquVg0_c009/gt/gt.txt'

gt = pd.read_csv(gt_path, names=['frame', 'person_id', 'x1', 'y1', 'w', 'h', 'temp1', 'temp2', 'temp3'])
gt = gt.drop(['temp1', 'temp2', 'temp3'], axis=1)
gt[((gt['frame'] == 1) | (gt['frame'] == 2)) & (gt['person_id'] == 6)].head()
gt[((gt['frame'] == 1) | (gt['frame'] == 2)) & (gt['person_id'] == 6)]['w'].max()

player_id = 10
max_w = gt[gt['person_id'] == player_id]['w'].max()
max_h = gt[gt['person_id'] == player_id]['h'].max()
dest_w = 224
dest_h = 224


In [3]:
rois = []
progress = tqdm(range(1, 488)[:64:2])
for frame in progress:
    formatted_path = img_path.format(frame=frame)
    img = cv2.imread(formatted_path)

    labels = [Label(*rec) for rec in gt[(gt['frame'] == frame) & (gt['person_id'] == player_id)].to_records()]
    # bboxes = np.array([[label.x1, label.y1, label.x1 + label.w, label.y1 + label.h] for label in labels])
    bboxes = np.array([[label.x1, label.y1, label.x1 + max_w, label.y1 + max_h] for label in labels])
    label = labels[0]
    roi = img[label.y1: label.y1 + max_h, label.x1: label.x1 + max_w]
    roi = cv2.resize(roi, (dest_w, dest_h), interpolation=cv2.INTER_AREA)
    # if roi.shape[0] < max_w:

    # roi= np.pad(roi, [(0,max(0, 244 - roi.shape[0])), (0,max(0, 244 - roi.shape[1])), (0,0)], constant_values=[[255],[255],[255]])
    # print(label.y1 + max_h, roi.shape)
    progress.set_postfix({'img': Path(formatted_path).name, 'roi': roi.shape})
    rois.append(roi)
    scores = mx.nd.array(np.array([np.ones(bboxes.shape[0])[:, None]]), ctx=gpu)
    classes = mx.nd.array(np.array([np.zeros(bboxes.shape[0])[:, None]]), ctx=gpu)
    det_labels = mx.nd.array(np.array([label.person_id for label in labels]), ctx=gpu)
    bboxes = mx.nd.array(np.array([bboxes[:]]), ctx=gpu)
    colors = {label.person_id: id_to_random_color(label.person_id, normalized=True) for label in labels}
    ax = utils.viz.cv_plot_bbox(img, bboxes[0], scores[0], labels=det_labels, colors=colors)
    cv2.imshow('frame', roi)
    cv2.waitKey(0)
    # if cv2.waitKey(1) == ord('q'):
    #     break
cv2.destroyAllWindows()
cv2.waitKey(1)
print(len(rois), 'x', rois[0].shape)

  0%|          | 0/32 [00:00<?, ?it/s, img=000001.jpg, roi=(224, 224, 3)][18:39:55] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8600 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.
QObject::moveToThread: Current thread (0x7d6dce0) is not the object's thread (0x487a76e0).
Cannot move to target thread (0x7d6dce0)

QObject::moveToThread: Current thread (0x7d6dce0) is not the object's thread (0x487a76e0).
Cannot move to target thread (0x7d6dce0)

QObject::moveToThread: Current thread (0x7d6dce0) is not the object's thread (0x487a76e0).
Cannot move to target thread (0x7d6dce0)

QObject::moveToThread: Current thread (0x7d6dce0) is not the object's thread (0x487a76e0).
Cannot move to target thread (0x7d6dce0)

QObject::moveToThread: Current thread (0x7d6dce0) is not the object's thread (0x487a76e0).
Cannot move to target thread (0x7d6dce0)

QObject::moveToThread: Current thread (0x7d6dce0) is not the object's thread (0x487a76e0).
Cann

32 x (224, 224, 3)


In [19]:
from models.i3d import I3D
i3d = I3D(feat_ext=True)
w = np.random.rand(1, 1024)
k = np.random.rand(1, 1024)
x = i3d.inference(rois).copyto(mx.cpu()).as_np_ndarray()
y = i3d.inference(rois).copyto(mx.cpu()).as_np_ndarray()
c = np.concatenate((x,y))

In [5]:
feat = np.load('./features/i3d_inceptionv1_kinetics400_46___cf3e328ce61448308b9e9ff3517b4aad.mp4_feat.npy')
feat.shape

FileNotFoundError: [Errno 2] No such file or directory: './features/i3d_inceptionv1_kinetics400_46___cf3e328ce61448308b9e9ff3517b4aad.mp4_feat.npy'